In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.manifold import TSNE

from nltk.stem.porter import *
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import wordnet 

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

import gensim

from google.cloud import bigquery

[nltk_data] Downloading package wordnet to /home/traffic/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/traffic/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/traffic/.local/lib/python3.8/site-packages/gensim/matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


In [3]:
# authenticate with BigQuery API
client = bigquery.Client.from_service_account_json('../../datascience-abovezero-58d98dcf7f65.json')
# Perform a query.
QUERY = ('SELECT * FROM `datascience-abovezero.ml_sandbox.chegg_influencers_comments`')
query_job = client.query(QUERY)  # API request
df =  query_job.result().to_dataframe() #transform to Pandas Dataframe

#df2 = pd.read_csv('analisis_comments_tiktok.csv')# Parameters tuning using Grid Search

# Bag Of Words

### Preprocessing 

In [4]:
def spelling_correcter(text):
    

def lemmatize_stemming(text):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

comments = df.comment.tolist()
comments_prep = [preprocess(str(comment)) for comment in comments]
comments_dict = gensim.corpora.Dictionary(comments_prep)
bow_corpus = [comments_dict.doc2bow(doc) for doc in comments_prep]

In [5]:
'''
Checking dictionary created
'''
count = 0
for k, v in comments_dict.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 ayyyi
1 legend
2 call
3 colleg
4 univers
5 amaz
6 person
7 antedragz
8 chegg
9 fuck
10 gift


### Training the model

### Testing the model

# TFIDF 

### Preprocessing

In [6]:
#docs_raw = df.comment.fillna('').tolist()

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 0.01)
#dtm_tf = tf_vectorizer.fit_transform(docs_raw)
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
#dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)

### Training The Model 

In [7]:
def get_top_topics(model, vectorizer, topn=5):
    names = tf_vectorizer.get_feature_names()
    res = {}
    for i_cluster, cluster in enumerate(model.components_):
        res[i_cluster]=[]
        for i_feature in cluster.argsort()[:-topn - 1:-1]:
            res[i_cluster] = res[i_cluster] + [names[i_feature]]
    return res

In [20]:
for i_infl, infl in enumerate(df.influencer.unique()):
    dfi = df.loc[df.influencer==infl,:]
    docs_raw = dfi.comment.fillna('').tolist()
    dtm_tf = tf_vectorizer.fit_transform(docs_raw)
    # train model to find topics
    lda_tf = LatentDirichletAllocation(n_components=4, random_state=0)
    lda_tf.fit(dtm_tf)
    # extract most important topics
    topics =  get_top_topics(lda_tf, tf_vectorizer, topn=5)
    print(infl)
    print(topics)
    
    X_test = tf_vectorizer.transform(dfi.loc[:,'comment'])
    doc_topic_dist_unnormalized = np.matrix(lda_tf.transform(X_test))
    for comment, comment_orig in zip(doc_topic_dist_unnormalized, dfi.comment):
        print(comment_orig)
        print(topics[comment[0].argmax()])
    if i_infl>-1:
        break

itssozer
{0: ['happy', 'best', 'nice', 'uni', 'makes'], 1: ['educate', 'antedragz', 'gift', 'like', 'itssozer'], 2: ['college', 'uni', 'person', 'know', 'different'], 3: ['uni', 'day', 'reply', 'does', 'university']}
Ayyyy legend
['happy', 'best', 'nice', 'uni', 'makes']
Are you serious
['happy', 'best', 'nice', 'uni', 'makes']
Yes it's college but it's called university
['happy', 'best', 'nice', 'uni', 'makes']
Your such an amazing person 😁
['happy', 'best', 'nice', 'uni', 'makes']
if i got chegg as a gift i would fucking die @Antedragz
['educate', 'antedragz', 'gift', 'like', 'itssozer']
That’s a different thing in my country tho
['college', 'uni', 'person', 'know', 'different']
：Ce gars est une legende 👏
['educate', 'antedragz', 'gift', 'like', 'itssozer']
Omg your so nice
['happy', 'best', 'nice', 'uni', 'makes']
What a gift
['educate', 'antedragz', 'gift', 'like', 'itssozer']
HEAVEN NOW
['happy', 'best', 'nice', 'uni', 'makes']
I love your page @itssozer 🥰😁
['happy', 'best', 'nice

In [11]:
df.head()

,date_post,date_extraction,influencer,post_type,post_url,platform,comment,comment_likecount,replies,BU
0,2021-9-2,2021-12-01,itssozer,VIDEO,https://www.tiktok.com/@itssozer/video/7003502...,TikTok,Ayyyy legend,50,None,CS
1,2021-9-2,2021-12-01,itssozer,VIDEO,https://www.tiktok.com/@itssozer/video/7003502...,TikTok,Are you serious,8,None,CS
2,2021-9-2,2021-12-01,itssozer,VIDEO,https://www.tiktok.com/@itssozer/video/7003502...,TikTok,Yes it's college but it's called university,9,None,CS
3,2021-9-2,2021-12-01,itssozer,VIDEO,https://www.tiktok.com/@itssozer/video/7003502...,TikTok,Your such an amazing person 😁,33,None,CS
4,2021-9-2,2021-12-01,itssozer,VIDEO,https://www.tiktok.com/@itssozer/video/7003502...,TikTok,if i got chegg as a gift i would fucking die @...,0,None,CS
